# Vanilla Gan

In [55]:
# Code by yunjey/pytorch-tutorial

## workflow

1. 실제 이미지들과 fake 이미지들을 샘플링합니다.


실제 이미지는 데이터셋에서 Load합니다.
fake 이미지는 Generator에 noise라는 인풋을 넣어서 만듭니다. 




2 . Discriminator를 학습시킵니다.

(1) 실제 이미지들을 넣고 분류기를 돌려봅니다.

real_loss: 실제 이미지들을 넣은 결과값들(0 혹은 1로 구성된 벡터)와 실제 이미지들의 레이블(1로 이루어진 벡터)를 비교해서 계산된 loss

(2) fake 이미지들을 넣고 분류기를 돌려봅니다.

fake_loss: fake 이미지들을 넣은 결과값(0 혹은 1로 구성된 벡터)와 fake 이미지들의 레이블(영벡터)를 비교해서 계산된 loss

(3) Discriminator's loss = real_loss + fake_loss


(4) 오차 역전파 및 파라미터 업데이트 

3 . Generator를 학습시킵니다.


(1) 새로운 fake 이미지들을 뽑아서 Discriminator에 일종의 테스트 셋으로 넣어봅니다.

fake 이미지는 역시 Generator에 noise를 넣어서 만듭니다.

(2) 테스트 결과값과 실제 이미지의 레이블을 비교해 loss를 계산합니다.


(3) 오차 역전파 및 파라미터 업데이트

4 . Generator가 만든 fake 이미지를 저장합니다.


### 1. 모듈 불러오기 및 환경세팅

In [3]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image


In [27]:
## Device configuration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 2. 하이퍼 파라미터 지정 및 데이터 로드

In [37]:
# Hyper-parameters
latent_size = 64 # z size , input size
hidden_size = 256
image_size = 784 # 28 * 28
num_epochs = 20 # 트레인 돌릴 에폭수 
batch_size = 100 # 배치 사이즈
sample_dir = 'samples'

In [38]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [39]:
# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
                                  std=(0.5, 0.5, 0.5))])


In [40]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

In [41]:
# Data loader
# dataset, 배치사이즈 , one-hot 여부 등등 지정한다.
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

### 3. D,G network 정의 loss 함수 정의

In [42]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),   #  size가 64인 noise를 인풋으로 받는다, 가중치 벡터를 곱해 256차원 벡터로 확장
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

In [43]:
# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size), # real image size만큼 shape 맞춘다. 
    nn.Tanh())

In [44]:
# Device setting
D = D.to(device)
G = G.to(device)

In [45]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss() # -> class real : 1 , fake :0
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [46]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [47]:
# grad 초기화 함수
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

### 4. 모델 트레이닝

In [48]:
# Start training
total_step = len(data_loader) # data_loader 길이 만큼 step_size 지정 

for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device) # cpu or gpu 
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #
        
        # D의 학습은 binary cross entopy 를 통해 진행
        
        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels) # loss 계산 (criterion = nn.BCELoss())
        real_score = outputs # real 이미지를 넣었을때의 스코어
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0

        # G의 인풋 공간  latent space 크기 만큼 노이즈 z 를 생성
        z = torch.randn(batch_size, latent_size).to(device)
        
        # 노이즈 z 를 G에 넣어 페이크 이미지 생성
        fake_images = G(z)
        
        # D 가 페이크 이미지를 판단한 결과
        outputs = D(fake_images)
        
        # 로스 계산
        d_loss_fake = criterion(outputs, fake_labels)
        
        # 위에서 D가 페이크 이미지에 대한 스코어 계산한 outputs 이 fake image 에 대한 스코어(0~1) 
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step() # back prob 결과를 업데이트 
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        # G의 인풋 공간  latent space 크기 만큼 노이즈 z 를 생성

        z = torch.randn(batch_size, latent_size).to(device)
        
        # G에 z를 넣어 페이크 이미지 생성
        fake_images = G(z)
        
        # 페이크 이미지를 판별
        outputs = D(fake_images)
        
        # G는 크로스 엔트로피 loss 계산 
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # 첫 에폭에서 real image 경로에 저장
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # 그 다음 에폭 돌때마다 fake image save
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# 모델 저장 ckpt or pickle 
# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch [0/20], Step [200/600], d_loss: 0.0475, g_loss: 3.9757, D(x): 0.99, D(G(z)): 0.04
Epoch [0/20], Step [400/600], d_loss: 0.0612, g_loss: 5.2755, D(x): 0.99, D(G(z)): 0.05
Epoch [0/20], Step [600/600], d_loss: 0.0546, g_loss: 5.4928, D(x): 0.98, D(G(z)): 0.03
Epoch [1/20], Step [200/600], d_loss: 0.0807, g_loss: 4.7682, D(x): 0.99, D(G(z)): 0.06
Epoch [1/20], Step [400/600], d_loss: 0.2162, g_loss: 3.5416, D(x): 0.99, D(G(z)): 0.17
Epoch [1/20], Step [600/600], d_loss: 0.5069, g_loss: 5.3590, D(x): 0.87, D(G(z)): 0.18
Epoch [2/20], Step [200/600], d_loss: 0.6882, g_loss: 3.3532, D(x): 0.75, D(G(z)): 0.07
Epoch [2/20], Step [400/600], d_loss: 0.3443, g_loss: 3.2709, D(x): 0.90, D(G(z)): 0.17
Epoch [2/20], Step [600/600], d_loss: 0.8237, g_loss: 2.0928, D(x): 0.70, D(G(z)): 0.22
Epoch [3/20], Step [200/600], d_loss: 0.3467, g_loss: 2.8760, D(x): 0.88, D(G(z)): 0.13
Epoch [3/20], Step [400/600], d_loss: 0.6924, g_loss: 2.4846, D(x): 0.87, D(G(z)): 0.30
Epoch [3/20], Step [600/600], d_